In [1]:
import glob
import pandas as pd
import os
paths = glob.glob("./calib_challenge_valdag_fork/unlabeled/*.avi")
current_file_index = 4
video_path = paths[current_file_index]
video_path

'./calib_challenge_valdag_fork/unlabeled/7.avi'

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
from torchvision.io import read_video
from torchvision.models.optical_flow import Raft_Small_Weights, raft_small
from tqdm import tqdm

def preprocess(img1_batch, img2_batch):
    img1_batch = F.resize(img1_batch, size=[520, 960], antialias=False)
    img2_batch = F.resize(img2_batch, size=[520, 960], antialias=False)
    return transforms(img1_batch, img2_batch)

In [3]:
frames, _, _ = read_video(str(video_path), output_format="TCHW")

weights = Raft_Small_Weights.DEFAULT
transforms = weights.transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = raft_small(weights=Raft_Small_Weights.DEFAULT, progress=False).to(device)
model = model.eval()

all_predicted_flows = []

for i in tqdm(range(len(frames) - 1)):
    img1_batch = torch.stack([frames[i]])
    img2_batch = torch.stack([frames[i+1]])
    img1_batch, img2_batch = preprocess(img1_batch, img2_batch)

    list_of_flows = model(img1_batch.to(device), img2_batch.to(device))
    predicted_flows = list_of_flows[-1]
    all_predicted_flows.append(predicted_flows.detach().cpu().resolve_conj().resolve_neg())
    
#     free up cuda space
    del list_of_flows
    del predicted_flows
    torch.cuda.empty_cache()

/home/s0001638/miniconda3/lib/python3.9/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1199/1199 [02:06<00:00,  9.49it/s]


In [4]:
all_predicted_flows = torch.stack(all_predicted_flows)
output_tensor = torch.mean(all_predicted_flows, dim=(3, 4), keepdim=True).squeeze()
output_tensor.shape

torch.Size([1199, 2])

In [5]:
filename = os.path.basename(paths[current_file_index])
output_file = filename.replace(".avi", ".pt")
output_file

'7.pt'

In [6]:
torch.save(output_tensor, "./raft_embeddings/"+output_file)